In [37]:
import tushare as ts
import pandas as pd
from copy import deepcopy
import numpy as np
# ts.set_token('f9d25f4ab3f0abe5e04fdf76c32e8c8a5cc94e384774da025098ec6e')
ts.set_token('60b6737f6c928ce9ebbfaba3966ab0f890d8984c4361211a164bb59f')
pro = ts.pro_api()
df = pro.user(token='60b6737f6c928ce9ebbfaba3966ab0f890d8984c4361211a164bb59f')
print(df)

   user_id        到期时间    到期积分
0   542060  2024-10-03  2000.0
1   542060  2024-09-22  3000.0


# 一、获取基金数据
1.从fund_basic获取场内基金基础信息
2.按照1中的代码，从fund_share获取基金规模，并按照规模排序
3.按照生成的基金规模排序表，手动选择不同品类的ETF，预计选择30+
4.从fund_daily获取ETF成立至今的行情

In [8]:
#提取基金列表和基金规模数据
df1 = pro.query('fund_basic',market='E')
df2 = pro.query('fund_share',trade_date='20240115')
print(df1)
print(df2)
#以ts_code建立索引
df1 = df1.set_index('ts_code')
df2 = df2.set_index('ts_code')
#以ts_code为索引将两个df合并
merged_df = pd.merge(df1, df2, on='ts_code', how='left')
print(merged_df)
#以规模份额降序
merged_df = merged_df.sort_values(by='fd_share', ascending=False)
merged_df.to_csv('fund_basic.csv', encoding='utf-8-sig')
# 选择份额排前200的作为备选基金池，以ETF名去重
df_fund_pool = merged_df[0:199].drop_duplicates(subset='name',keep='first',inplace=False)

        ts_code       name management custodian fund_type found_date  \
0     513870.SH  纳斯达克指数ETF       富国基金      招商银行       股票型   20231025   
1     517520.SH    N黄金股ETF       永赢基金    中国农业银行       股票型   20231024   
2     561360.SH      石油ETF       国泰基金  中国邮政储蓄银行       股票型   20231023   
3     159520.SZ    消费龙头ETF     工银瑞信基金      浦发银行       股票型   20231019   
4     560280.SH    工程机械ETF       广发基金      招商银行       股票型   20231018   
...         ...        ...        ...       ...       ...        ...   
1865  500019.SH       基金普润       鹏华基金    中国工商银行       股票型   19920509   
1866  500013.SH       基金安瑞       华安基金    中国工商银行       股票型   19920429   
1867  500017.SH       基金景业       大成基金    中国农业银行       股票型   19920401   
1868  184702.SZ       基金同智       长盛基金      中国银行       股票型   19920313   
1869  500028.SH       基金兴业       华夏基金    中国农业银行       股票型   19911115   

      due_date list_date issue_date delist_date  ...  min_amount  exp_return  \
0         None  20231102   20231016        None  ...   

In [38]:
ts.pro_bar(ts_code='588000.SH',asset='FD',adj='qfq',freq='D')

,ts_code,trade_date,pre_close,open,high,low,close,change,pct_chg,vol,amount
0,588000.SH,20240202,0.726,0.726,0.729,0.686,0.707,-0.019,-2.6171,52639701.88,3739941.703
1,588000.SH,20240201,0.722,0.716,0.741,0.714,0.726,0.004,0.5540,41918586.88,3054122.051
2,588000.SH,20240131,0.735,0.731,0.736,0.717,0.722,-0.013,-1.7687,46771777.22,3401154.458
3,588000.SH,20240130,0.764,0.757,0.760,0.732,0.735,-0.029,-3.7958,40231346.42,3000318.362
4,588000.SH,20240129,0.785,0.791,0.794,0.763,0.764,-0.021,-2.6752,36301416.31,2815107.805
...,...,...,...,...,...,...,...,...,...,...,...
780,588000.SH,20201120,1.438,1.436,1.444,1.431,1.435,-0.003,-0.2086,6010033.63,862623.917
781,588000.SH,20201119,1.424,1.424,1.440,1.412,1.438,0.014,0.9831,6854018.07,980394.625
782,588000.SH,20201118,1.436,1.435,1.445,1.422,1.424,-0.012,-0.8357,8631881.31,1234788.628
783,588000.SH,20201117,1.465,1.460,1.460,1.430,1.436,-0.029,-1.9795,10806365.35,1553721.900


# 二、实现一个demo
1.测试talib库，使用移动平均线
2.选择510300.SH沪深300ETF，根据均线形成信号，形成trade_info仓位权重，
3.使用backtrader进行回测

In [57]:
import talib as ta
df_daily = ts.pro_bar(ts_code='588000.SH',asset='FD',adj='qfq',freq='D').sort_values(by='trade_date',ascending=True)
df_daily = df_daily.rename(columns={'trade_date':'datetime', 'vol':'volume', 'ts_code':'sec_code'})
# df_daily['datetime'] = pd.to_datetime(df_daily['datetime'], format='%Y-%m-%d')
# CDL3BLACKCROWS = ta.CDL3BLACKCROWS(df_daily.open.values,df_daily.high.values,df_daily.low.values,df_daily.close.values)
# CDL3BLACKCROWS

In [58]:
df_daily['datetime'] = df_daily['datetime'].str.replace(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3')
df_daily = df_daily.assign(openinterest=0)
df_daily
# df_daily.drop(columns='ts_code')

,sec_code,datetime,pre_close,open,high,low,close,change,pct_chg,volume,amount,openinterest
784,588000.SH,2020-11-16,1.439,1.556,1.620,1.451,1.465,0.026,1.8068,14453638.37,2119240.683,0
783,588000.SH,2020-11-17,1.465,1.460,1.460,1.430,1.436,-0.029,-1.9795,10806365.35,1553721.900,0
782,588000.SH,2020-11-18,1.436,1.435,1.445,1.422,1.424,-0.012,-0.8357,8631881.31,1234788.628,0
781,588000.SH,2020-11-19,1.424,1.424,1.440,1.412,1.438,0.014,0.9831,6854018.07,980394.625,0
780,588000.SH,2020-11-20,1.438,1.436,1.444,1.431,1.435,-0.003,-0.2086,6010033.63,862623.917,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4,588000.SH,2024-01-29,0.785,0.791,0.794,0.763,0.764,-0.021,-2.6752,36301416.31,2815107.805,0
3,588000.SH,2024-01-30,0.764,0.757,0.760,0.732,0.735,-0.029,-3.7958,40231346.42,3000318.362,0
2,588000.SH,2024-01-31,0.735,0.731,0.736,0.717,0.722,-0.013,-1.7687,46771777.22,3401154.458,0
1,588000.SH,2024-02-01,0.722,0.716,0.741,0.714,0.726,0.004,0.5540,41918586.88,3054122.051,0


In [24]:
df_daily

,sec_code,datetime,pre_close,open,high,low,close,change,pct_chg,volume,amount,openinterest
783,588000.SH,2020-11-16,1.439,1.556,1.620,1.451,1.465,0.026,1.8068,14453638.37,2119240.683,0
782,588000.SH,2020-11-17,1.465,1.460,1.460,1.430,1.436,-0.029,-1.9795,10806365.35,1553721.900,0
781,588000.SH,2020-11-18,1.436,1.435,1.445,1.422,1.424,-0.012,-0.8357,8631881.31,1234788.628,0
780,588000.SH,2020-11-19,1.424,1.424,1.440,1.412,1.438,0.014,0.9831,6854018.07,980394.625,0
779,588000.SH,2020-11-20,1.438,1.436,1.444,1.431,1.435,-0.003,-0.2086,6010033.63,862623.917,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4,588000.SH,2024-01-26,0.804,0.800,0.802,0.781,0.785,-0.019,-2.3632,34189779.38,2709998.422,0
3,588000.SH,2024-01-29,0.785,0.791,0.794,0.763,0.764,-0.021,-2.6752,36301416.31,2815107.805,0
2,588000.SH,2024-01-30,0.764,0.757,0.760,0.732,0.735,-0.029,-3.7958,40231346.42,3000318.362,0
1,588000.SH,2024-01-31,0.735,0.731,0.736,0.717,0.722,-0.013,-1.7687,46771777.22,3401154.458,0


In [59]:
df_daily.reset_index(drop=True, inplace=True)

In [42]:
df_trade = pd.DataFrame(columns=['trade_date','sec_code','weight'])
sma = ta.SMA(df_daily.close,timeperiod=10)
sec_code='588000.SH'
for i in range(len(df_daily)):
    if not np.isnan(sma[i]):
        print(i)
        trade_date = df_daily['datetime'][i]
        if df_daily['close'][i] > sma[i]:
            weight = 1
        else:
            weight = 0
        if len(df_trade) > 0:
            if df_trade.iloc[-1]['weight'] != weight:
                df_trade.loc[len(df_trade)] = [trade_date,sec_code,weight]
                # print(df_trade.iloc[-1])
        else:
            df_trade.loc[len(df_trade)] = [trade_date,sec_code,weight]
            # print(df_trade.iloc[-1])
df_trade = df_trade.sort_values(by='trade_date',ascending=True)
df_trade.reset_index(drop=True, inplace=True)
df_trade['trade_date'] = df_trade['trade_date'].str.replace(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3')
print(df_trade)
            


9
trade_date    2020-11-27
sec_code       588000.SH
weight                 0
Name: 0, dtype: object
10
11
12
trade_date    2020-12-02
sec_code       588000.SH
weight                 1
Name: 1, dtype: object
13
14
15
16
17
trade_date    2020-12-09
sec_code       588000.SH
weight                 0
Name: 2, dtype: object
18
19
20
trade_date    2020-12-14
sec_code       588000.SH
weight                 1
Name: 3, dtype: object
21
22
trade_date    2020-12-16
sec_code       588000.SH
weight                 0
Name: 4, dtype: object
23
trade_date    2020-12-17
sec_code       588000.SH
weight                 1
Name: 5, dtype: object
24
25
26
27
28
trade_date    2020-12-24
sec_code       588000.SH
weight                 0
Name: 6, dtype: object
29
30
31
32
trade_date    2020-12-30
sec_code       588000.SH
weight                 1
Name: 7, dtype: object
33
34
35
36
37
38
trade_date    2021-01-08
sec_code       588000.SH
weight                 0
Name: 8, dtype: object
39
trade_date    2021-01-11
s

In [43]:
df_trade

,trade_date,sec_code,weight
0,2020-11-27,588000.SH,0
1,2020-12-02,588000.SH,1
2,2020-12-09,588000.SH,0
3,2020-12-14,588000.SH,1
4,2020-12-16,588000.SH,0
...,...,...,...
118,2023-12-14,588000.SH,0
119,2023-12-28,588000.SH,1
120,2024-01-03,588000.SH,0
121,2024-01-25,588000.SH,1


In [44]:
df_trade.to_csv('trade_info_sma.csv')

In [60]:

df_daily['datetime'] = pd.to_datetime(df_daily['datetime'])
df_daily = df_daily.set_index('datetime')

In [62]:
df_daily.index

DatetimeIndex(['2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19',
               '2020-11-20', '2020-11-23', '2020-11-24', '2020-11-25',
               '2020-11-26', '2020-11-27',
               ...
               '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25',
               '2024-01-26', '2024-01-29', '2024-01-30', '2024-01-31',
               '2024-02-01', '2024-02-02'],
              dtype='datetime64[ns]', name='datetime', length=785, freq=None)

In [63]:
import backtrader as bt
class PandasData(bt.feed.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''

    # params = (
    #     # Possible values for datetime (must always be present)
    #     #  None : datetime is the "index" in the Pandas Dataframe
    #     #  -1 : autodetect position or case-wise equal name
    #     #  >= 0 : numeric index to the colum in the pandas dataframe
    #     #  string : column name (as index) in the pandas dataframe
    #     ('datetime', 'trade_date'),

    #     # Possible values below:
    #     #  None : column not present
    #     #  -1 : autodetect position or case-wise equal name
    #     #  >= 0 : numeric index to the colum in the pandas dataframe
    #     #  string : column name (as index) in the pandas dataframe
    #     ('open', 'open'),
    #     ('high', 'high'),
    #     ('low', 'low'),
    #     ('close', 'close'),
    #     ('volume', 'vol'),
    #     ('openinterest', None),
    # )
class TestStrategy(bt.Strategy):
    
    params = (
        ('buy_stocks', None), # 传入各个调仓日的股票列表和相应的权重
    )
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('{}, {}'.format(dt.isoformat(), txt))

    def __init__(self):
         # 读取调仓日期，回测时，会在这一天下单，然后在下一个交易日，以开盘价买入
        # self.trade_dates = pd.to_datetime(self.p.buy_stocks['trade_date'].unique()).tolist()
        # self.buy_stock = self.p.buy_stocks # 保留调仓信息
        
        self.trade_dates = pd.to_datetime(df_trade['trade_date'].unique()).tolist()
        self.buy_stock = df_trade
        
        self.order_list = []  # 记录以往订单，在调仓日要全部取消未成交的订单
        self.buy_stocks_pre = [] # 记录上一期持仓
    
    def next(self):
        # 获取当前的回测时间点
        dt = self.datas[0].datetime.date(0)
        # 打印当前时刻的总资产
        self.log('当前总资产 %.2f' %(self.broker.getvalue()))
        # 如果是调仓日，则进行调仓操作
        if dt in self.trade_dates:
            print("--------------{} 为调仓日----------".format(dt))
            #取消之前所下的没成交也未到期的订单
            if len(self.order_list) > 0:
                print("--------------- 撤销未完成的订单 -----------------")
                for od in self.order_list:
                    # 如果订单未完成，则撤销订单
                    self.cancel(od) 
                 #重置订单列表
                self.order_list = [] 
                
            # 提取当前调仓日的持仓列表
            buy_stocks_data = self.buy_stock.query(f"trade_date=='{dt}'")
            long_list = buy_stocks_data['sec_code'].tolist()
            print('long_list', long_list)  # 打印持仓列表
            
            # 对现有持仓中，调仓后不再继续持有的股票进行卖出平仓#trade_info中记录的是每个调仓日权重，如果股票权重为0则不会单独标明，因此不在记录中的为清仓股票
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock)
            if len(sell_stock) > 0:
                print("-----------对不再持有的股票进行平仓--------------")
                for stock in sell_stock:
                    data = self.getdatabyname(stock)
                    if self.getposition(data).size > 0 :
                        od = self.close(data=data)  
                        self.order_list.append(od) # 记录卖出订单

            # 买入此次调仓的股票：多退少补原则
            print("-----------买入此次调仓期的股票--------------")
            for stock in long_list:
                w = buy_stocks_data.query(f"sec_code=='{stock}'")['weight'].iloc[0] # 提取持仓权重
                data = self.getdatabyname(stock)
                order = self.order_target_percent(data=data, target=w*0.95) # 为减少可用资金不足的情况，留 5% 的现金做备用
                self.order_list.append(order)
                
            self.buy_stocks_pre = long_list  # 保存此次调仓的股票列表
        
    #订单日志    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.value,
                     order.executed.comm,
                     order.executed.size,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref,
                         order.executed.price,
                         order.executed.value,
                         order.executed.comm,
                         order.executed.size,
                         order.data._name))
        

In [64]:
# 实例化大脑
cerebro_ = bt.Cerebro() 

# 按股票代码，依次循环传入数据
for stock in df_daily['sec_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=df_daily.index.unique())
    df = df_daily.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)
    datafeed1 = bt.feeds.PandasData(dataname=data_)
    cerebro_.adddata(datafeed1, name=stock)
    print(f"{stock} Done !") 

cerebro_.addstrategy(TestStrategy)


588000.SH Done !


0

In [50]:
df_trade['trade_date'] = pd.to_datetime(df_trade['trade_date'], format='%Y-%m-%d')
df_trade['trade_date']

0     2020-11-27
1     2020-12-02
2     2020-12-09
3     2020-12-14
4     2020-12-16
         ...    
118   2023-12-14
119   2023-12-28
120   2024-01-03
121   2024-01-25
122   2024-01-26
Name: trade_date, Length: 123, dtype: datetime64[ns]

In [56]:
df_trade1 = pd.to_datetime(df_trade['trade_date'].unique()).tolist()

[Timestamp('2020-11-27 00:00:00'),
 Timestamp('2020-12-02 00:00:00'),
 Timestamp('2020-12-09 00:00:00'),
 Timestamp('2020-12-14 00:00:00'),
 Timestamp('2020-12-16 00:00:00'),
 Timestamp('2020-12-17 00:00:00'),
 Timestamp('2020-12-24 00:00:00'),
 Timestamp('2020-12-30 00:00:00'),
 Timestamp('2021-01-08 00:00:00'),
 Timestamp('2021-01-11 00:00:00'),
 Timestamp('2021-01-27 00:00:00'),
 Timestamp('2021-02-09 00:00:00'),
 Timestamp('2021-02-22 00:00:00'),
 Timestamp('2021-03-22 00:00:00'),
 Timestamp('2021-03-23 00:00:00'),
 Timestamp('2021-03-26 00:00:00'),
 Timestamp('2021-04-09 00:00:00'),
 Timestamp('2021-04-16 00:00:00'),
 Timestamp('2021-05-06 00:00:00'),
 Timestamp('2021-05-14 00:00:00'),
 Timestamp('2021-06-16 00:00:00'),
 Timestamp('2021-06-17 00:00:00'),
 Timestamp('2021-07-02 00:00:00'),
 Timestamp('2021-07-05 00:00:00'),
 Timestamp('2021-07-06 00:00:00'),
 Timestamp('2021-07-07 00:00:00'),
 Timestamp('2021-07-14 00:00:00'),
 Timestamp('2021-07-21 00:00:00'),
 Timestamp('2021-07-

In [65]:
cerebro = deepcopy(cerebro_)  # 深度复制已经导入数据的 cerebro_，避免重复导入数据 
# 初始资金 100,000,000    
cerebro.broker.setcash(100000000.0) 
# 添加策略
cerebro.addstrategy(TestStrategy, buy_stocks=df_trade) # 通过修改参数 buy_stocks ，使用同一策略回测不同的持仓列表

# 添加分析指标
# 返回年初至年末的年度收益率
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
# 计算最大回撤相关指标
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
# 计算年化收益
cerebro.addanalyzer(bt.analyzers.Returns, _name='_Returns', tann=252)
# 计算年化夏普比率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio', timeframe=bt.TimeFrame.Days, annualize=True, riskfreerate=0) # 计算夏普比率
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='_SharpeRatio_A')
# 返回收益率时序
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')

# 启动回测
result = cerebro.run()

2020-11-16, 当前总资产 100000000.00
2020-11-16, 当前总资产 100000000.00
2020-11-17, 当前总资产 100000000.00
2020-11-17, 当前总资产 100000000.00
2020-11-18, 当前总资产 100000000.00
2020-11-18, 当前总资产 100000000.00
2020-11-19, 当前总资产 100000000.00
2020-11-19, 当前总资产 100000000.00
2020-11-20, 当前总资产 100000000.00
2020-11-20, 当前总资产 100000000.00
2020-11-23, 当前总资产 100000000.00
2020-11-23, 当前总资产 100000000.00
2020-11-24, 当前总资产 100000000.00
2020-11-24, 当前总资产 100000000.00
2020-11-25, 当前总资产 100000000.00
2020-11-25, 当前总资产 100000000.00
2020-11-26, 当前总资产 100000000.00
2020-11-26, 当前总资产 100000000.00
2020-11-27, 当前总资产 100000000.00
--------------2020-11-27 为调仓日----------
long_list ['588000.SH']
sell_stock []
-----------买入此次调仓期的股票--------------
2020-11-27, 当前总资产 100000000.00
--------------2020-11-27 为调仓日----------
long_list ['588000.SH']
sell_stock []
-----------买入此次调仓期的股票--------------
2020-11-30, 当前总资产 100000000.00
2020-11-30, 当前总资产 100000000.00
2020-12-01, 当前总资产 100000000.00
2020-12-01, 当前总资产 100000000.00
2020-12-02, 当前总资产 100000000.

In [68]:
cerebro.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>], [<Figure size 640x480 with 4 Axes>]]

# 二、增加日常观测指标
1.使用talib库，筛选移动平均线/MACD/布林带右侧标的，或研究其他趋势特征
2.构建talib指标库，筛选单一/组合技术指标标的，如反复阴阳、光头阳线

# 三、策略规范化
1.使用组合，构建时序规则模型
2.品种风险衡量，如相关性
3.资金分配，比如风险平价
4.设置止损/止盈条件，综合考虑成本/技术指标/趋势
5.考虑使用因子投资手册方法论分析因子

# 四、策略优化
1.尝试使用期货/期指构建CTA时序规则或截面模型
2.尝试使用机器学习构建时序预测模型

In [67]:
a = [1,2,3]
b = [3,1,2]
c = a>b
c

False